### Import Dataset

In [1]:
# Load data from the dataset file
# Divide the dsatavin train, validate and test to train the model and to see its accuracy. The val part is used for parameters adjusting like to find better initialization of the modles

import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from enum import Enum
import torch
from typing import Tuple, Callable, Optional, List
from dataclasses import dataclass
from mashumaro import DataClassDictMixin
from loguru import logger
 
# Enum for different types of validation splits
class ValidationSplit(Enum):
    # Temporal Holdout: Reserve the last part of time-ordered data for validation
    TEMPORAL_HOLDOUT = "temporal-holdout"
    "Reserve the last portion (e.g., 10-20%) of your time-ordered data for validation, and use the remaining data for training. This is a simple and widely used approach."

# Custom Dataset class for handling pandas DataFrame for time series
class DataframeDataset(Dataset):
    """Dataset from a pandas dataframe
    """    

    def __init__(self, df: pd.DataFrame, window_size_input: int, window_size_predict: int, transform: Optional[Callable] = None):
        """Constructor
        # Constructor for initializing dataset with DataFrame and window sizes

        Args:
            df (pd.DataFrame): Dataframe
            window_size_input (int): Input window size
            window_size_predict (int): Prediction window size
            transform (Optional[Callable], optional): Transforms such as normalization applied to time series. Defaults to None.
        """        
        window_size_total = window_size_input + window_size_predict
        assert len(df) > window_size_total, f"Dataset length ({len(df)}) must be greater than window size ({window_size_total})"
        self.df = df
        self.window_size_input = window_size_input
        self.window_size_predict = window_size_predict
        self.transform = transform

    def __len__(self):
        return len(self.df) - self.window_size_input - self.window_size_predict
        # Returns the length of the dataset adjusted for window sizes

    def get_sample(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """Get a window sample. Input from [idx, idx + window_size_input], prediction from [idx + window_size_input, idx + window_size_input + window_size_predict

        Args:
            idx (int): Index

        Returns:
            Tuple[torch.Tensor, torch.Tensor]: Input and prediction tensors
        """

        # Check if the index plus window size exceeds the length of the dataset
        if idx + self.window_size_input + self.window_size_predict > len(self.df):
            raise IndexError(f"Index ({idx}) + window_size_input ({self.window_size_input}) + window_size_predict ({self.window_size_predict}) exceeds dataset length ({len(self.df)})")

        # Window the data
        sample_input = self.df.iloc[idx:idx + self.window_size_input, :]
        sample_pred = self.df.iloc[idx + self.window_size_input:idx + self.window_size_input + self.window_size_predict, :]

        # Convert to torch tensor
        sample_input = torch.tensor(sample_input.values, dtype=torch.float32)
        sample_pred = torch.tensor(sample_pred.values, dtype=torch.float32)

        # Apply transform
        if self.transform is not None:
            sample_input = self.transform(sample_input)
            sample_pred = self.transform(sample_pred)

        return sample_input, sample_pred

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if isinstance(idx, list):
            # Handle a list of indices
            samples = [self.get_sample(i) for i in idx]
            return samples
        else:
            # Handle a single index
            return self.get_sample(idx)


@dataclass
class DataNormalization(DataClassDictMixin):
    mean_each_feature: Optional[List[float]] = None
    "Mean for each feature"

    std_each_feature: Optional[List[float]] = None
    "Std for each feature"


def load_csv_dataset(
    csv_file: str, 
    batch_size: int, 
    input_length: int, 
    prediction_length: int, 
    val_split: ValidationSplit, 
    val_split_holdout: float = 0.2, 
    shuffle: bool = True,
    normalize_each_feature: bool = True,
    data_norm_exist: Optional[DataNormalization] = None
    ) -> Tuple[DataLoader, DataLoader, DataNormalization]:
    """Load a CSV dataset

    Args:
        csv_file (str): CSV file path
        batch_size (int): Batch size
        input_length (int): Input length
        prediction_length (int): Prediction length
        val_split (ValidationSplit): Validation split method
        val_split_holdout (float, optional): Holdout fraction for validation (last X% of data) - only used for TEMPORAL_HOLDOUT. Defaults to 0.2.
        shuffle (bool, optional): True to shuffle data. Defaults to True.
        normalize_each_feature (bool, optional): Normalize each feature. Defaults to True.
        data_norm_exist (Optional[DataNormalization], optional): Existing normalization data - apply this instead of recalculating. Defaults to None.

    Returns:
        Tuple[DataLoader, DataLoader, DataNormalization]: Training and validation data loaders, and normalization data
    """    

    # Load the CSV file into a DataFrame
    df_raw = pd.read_csv(csv_file)
    df = df_raw.set_index('date')

    # Make dataset
    dataset = DataframeDataset(df, window_size_input=input_length, window_size_predict=prediction_length)
    no_pts = len(dataset)

    # Split the data into training and validation
    if val_split == ValidationSplit.TEMPORAL_HOLDOUT:
        idx_train_val = int(no_pts * (1-val_split_holdout))
    else:
        raise NotImplementedError(f"Validation split {val_split} not implemented")

    # Normalize each feature separately        
    if data_norm_exist is None:
        data_norm_exist = DataNormalization()

        # Compute mean and std on training data from pandas dataframe
        filtered_df = df[:idx_train_val]
        data_norm_exist.mean_each_feature = list(filtered_df.mean().values)
        data_norm_exist.std_each_feature = list(filtered_df.std().values)
        logger.debug(f"Computed data mean for each feature: {data_norm_exist.mean_each_feature}")
        logger.debug(f"Computed data std for each feature: {data_norm_exist.std_each_feature}")

    if normalize_each_feature:
        assert data_norm_exist.mean_each_feature is not None, "Must provide data mean for each feature"
        assert data_norm_exist.std_each_feature is not None, "Must provide data std for each feature"

        # Create a normalization function
        transform = lambda x: (x - torch.Tensor(data_norm_exist.mean_each_feature)) / torch.Tensor(data_norm_exist.std_each_feature)

        # Apply the normalization function
        dataset.transform = transform

    # Splits
    train_dataset = Subset(dataset, range(idx_train_val))
    val_dataset = Subset(dataset, range(idx_train_val, no_pts))

    loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    loader_val = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)

    return loader_train, loader_val, data_norm_exist


C:\Users\geral\AppData\Local\Temp\ipykernel_25468\2861766496.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Model Layers

In [2]:
# In this file is the initialization of the model, how it works how is the input implemented in the model.
# It has all the layers which train the model and fit it with the data
# As we can see it has some classes for mixing layers and features and some other actions like transpèozation of data as time series mixer work.
# As we know they work by mixing feature and samples and training the model in 2 perspectives like feature dependency and time dependency.

import torch.nn as nn
import torch

class TSBatchNorm2d(nn.Module):
    def __init__(self):
        super(TSBatchNorm2d, self).__init__()
        self.bn = nn.BatchNorm2d(num_features=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = x.unsqueeze(1)
        output = self.bn(x)
        output = output.squeeze(1)
        return output

class TSTimeMixingResBlock(nn.Module):
    def __init__(self, width_time: int, dropout: float):
        super(TSTimeMixingResBlock, self).__init__()
        self.norm = TSBatchNorm2d()
        self.lin = nn.Linear(in_features=width_time, out_features=width_time)
        self.dropout = nn.Dropout(p=dropout)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.norm(x)
        y = torch.transpose(y, 1, 2)
        y = self.lin(y)
        y = self.act(y)
        y = torch.transpose(y, 1, 2)
        y = self.dropout(y)
        return x + y

class TSFeatMixingResBlock(nn.Module):
    def __init__(self, width_feats: int, width_feats_hidden: int, dropout: float):
        super(TSFeatMixingResBlock, self).__init__()
        self.norm = TSBatchNorm2d()
        self.lin_1 = nn.Linear(in_features=width_feats, out_features=width_feats_hidden)
        self.lin_2 = nn.Linear(in_features=width_feats_hidden, out_features=width_feats)
        self.dropout_1 = nn.Dropout(p=dropout)
        self.dropout_2 = nn.Dropout(p=dropout)
        self.act = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.norm(x)
        y = self.lin_1(y)
        y = self.act(y)
        y = self.dropout_1(y)
        y = self.lin_2(y)
        y = self.dropout_2(y)
        return x + y

class TSMixingLayer(nn.Module):
    def __init__(self, input_length: int, no_feats: int, feat_mixing_hidden_channels: int, dropout: float):
        super(TSMixingLayer, self).__init__()
        self.time_mixing = TSTimeMixingResBlock(width_time=input_length, dropout=dropout)
        self.feat_mixing = TSFeatMixingResBlock(width_feats=no_feats, width_feats_hidden=feat_mixing_hidden_channels, dropout=dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = self.time_mixing(x)
        y = self.feat_mixing(y)
        return y

class TSTemporalProjection(nn.Module):
    def __init__(self, input_length: int, forecast_length: int):
        super(TSTemporalProjection, self).__init__()
        self.lin = nn.Linear(in_features=input_length, out_features=forecast_length)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        y = torch.transpose(x, 1, 2)
        y = self.lin(y)
        y = torch.transpose(y, 1, 2)
        return y

class TSMixerModelExclRIN(nn.Module):
    def __init__(self, input_length: int, forecast_length: int, no_feats: int, feat_mixing_hidden_channels: int, no_mixer_layers: int, dropout: float):
        super(TSMixerModelExclRIN, self).__init__()
        self.temp_proj = TSTemporalProjection(input_length=input_length, forecast_length=forecast_length)
        mixer_layers = []
        for _ in range(no_mixer_layers):
            mixer_layers.append(TSMixingLayer(input_length=input_length, no_feats=no_feats, feat_mixing_hidden_channels=feat_mixing_hidden_channels, dropout=dropout))
        self.mixer_layers = nn.ModuleList(mixer_layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for mixer_layer in self.mixer_layers:
            x = mixer_layer(x)
        x = self.temp_proj(x)
        return x

class TSMixerModel(nn.Module):
    def __init__(self, input_length: int, forecast_length: int, no_feats: int, feat_mixing_hidden_channels: int, no_mixer_layers: int, dropout: float, eps: float = 1e-8):
        super(TSMixerModel, self).__init__()
        self.eps = eps
        self.scale = nn.Parameter(torch.ones(no_feats))
        self.shift = nn.Parameter(torch.zeros(no_feats))
        self.ts = TSMixerModelExclRIN(
            input_length=input_length,
            forecast_length=forecast_length,
            no_feats=no_feats,
            feat_mixing_hidden_channels=feat_mixing_hidden_channels,
            no_mixer_layers=no_mixer_layers,
            dropout=dropout
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mean = torch.mean(x, dim=1, keepdim=True)
        var = torch.var(x, dim=1, keepdim=True)
        x = (x - mean) / torch.sqrt(var + self.eps)
        x = x * self.scale + self.shift
        x = self.ts(x)
        x = (x - self.shift) / self.scale
        x = x * torch.sqrt(var + self.eps) + mean
        return x


### Model Initialization

In [3]:
from utils.tsmixer_conf import TSMixerConf, TrainingMetadata, makedirs
from utils.model import TSMixerModel
from utils.load_csv import DataNormalization

import os
from typing import Optional, Tuple, Dict, List
import torch
from loguru import logger
from tqdm import tqdm
import json
import time
import shutil
from dataclasses import dataclass
from mashumaro import DataClassDictMixin
import yaml


class TSMixer:
    """TSMixer including training and prediction methods
    """    


    def __init__(self, conf: TSMixerConf):
        """
        Constructor for TSMixer class

        Args:
            conf (TSMixerConf): Configuration
        """
        # Rest of your constructor code goes here
        self.conf = conf


        # Create the model
        self.model = TSMixerModel(
            input_length=self.conf.input_length,
            forecast_length=self.conf.prediction_length,
            no_feats=self.conf.no_features,
            feat_mixing_hidden_channels=self.conf.feat_mixing_hidden_channels or self.conf.no_features,
            no_mixer_layers=self.conf.no_mixer_layers,
            dropout=self.conf.dropout
            ) 

        # Move to device
        self.model.to(self.conf.device)

        # Load the model
        if self.conf.initialize == self.conf.Initialize.FROM_LATEST_CHECKPOINT:
            self.load_checkpoint(fname=self.conf.checkpoint_latest)
        elif self.conf.initialize == self.conf.Initialize.FROM_BEST_CHECKPOINT:
            self.load_checkpoint(fname=self.conf.checkpoint_best)
        elif self.conf.initialize == self.conf.Initialize.FROM_SCRATCH:
            pass
        else:
            raise NotImplementedError(f"Initialize {self.conf.initialize} not implemented")


    def load_checkpoint(self, fname: str, optimizer: Optional[torch.optim.Optimizer] = None) -> Tuple[int,float]:
        """Load a checkpoint, optionally including the optimizer state

        Args:
            fname (str): File name
            optimizer (Optional[torch.optim.Optimizer], optional): Optimizer to update from checkpoint. Defaults to None.

        Returns:
            Tuple[int,float]: Epoch and loss
        """        
        logger.debug(f"Loading model weights from {fname}")
        checkpoint = torch.load(fname)
        self.model.load_state_dict(checkpoint['model_state_dict'])

        if optimizer is not None:
            logger.debug(f"Loading optimizer state from {fname}")
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        logger.info(f"Loaded optimizer state from epoch {epoch} with loss {loss}")
        return epoch, loss


    def predict(self, batch_input: torch.Tensor) -> torch.Tensor:
        """Predict the output for a batch of input data

        Args:
            batch_input (torch.Tensor): Input data of shape (batch_size, input_length (time), no_features)

        Returns:
            torch.Tensor: Predicted output of shape (batch_size, prediction_length (time), no_features)
        """        
        self.model.eval()

        # Check size
        assert batch_input.shape[1] == self.conf.input_length, f"Input length {batch_input.shape[1]} does not match configuration {self.conf.input_length}"
        assert batch_input.shape[2] == self.conf.no_features, f"Number of features {batch_input.shape[2]} does not match configuration {self.conf.no_features}"

        # Predict
        batch_input = batch_input.to(self.conf.device)
        with torch.no_grad():
            batch_pred_hat = self.model(batch_input)
        return batch_pred_hat

    def load_data_norm(self) -> Optional[DataNormalization]:
        """Load the data normalization from a JSON file

        Returns:
            Optional[DataNormalization]: Data normalization, or None if the file does not exist
        """        

        if os.path.exists(self.conf.data_norm_json):
            logger.debug(f"Loading data normalization from {self.conf.data_norm_json}")
            with open(self.conf.data_norm_json, "r") as f:
                return DataNormalization.from_dict(json.load(f))
        else:
            return None


    @dataclass
    class PredData(DataClassDictMixin):
        """Prediction data
        """        

        pred_gt: List[List[float]]
        "Ground truth prediction"

        pred: List[List[float]]
        "Model prediction"

        inputs: Optional[List[List[float]]] = None
        "Inputs"


    def predict_val_dataset(self, max_samples: Optional[int] = None, save_inputs: bool = False) -> List[PredData]:
        """Predict on the validation dataset

        Args:
            max_samples (Optional[int], optional): Maximum number of samples to predict from the validation dataset. Defaults to None.
            save_inputs (bool, optional): Save the inputs as well as the predictions. Defaults to False.

        Returns:
            List[PredData]: List of predictions
        """        

        # Change batch size to 1 and not shuffle data for consistency
        batch_size_save = self.conf.batch_size
        shuffle_save = self.conf.shuffle
        self.conf.batch_size = 1
        self.conf.shuffle = False

        # Load the data normalization if it exists and use it
        data_norm = self.load_data_norm()

        # Create the loaders
        _, loader_val, _ = self.conf.create_data_loaders_train_val(data_norm)
        
        # Predict
        data_list: List[TSMixer.PredData] = []
        for _ in tqdm(range(max_samples or len(loader_val)), desc="Predicting"):
            batch_input, batch_pred = next(iter(loader_val))
            batch_pred_hat = self.predict(batch_input)
            data = TSMixer.PredData(
                pred_gt=batch_pred.tolist()[0],
                pred=batch_pred_hat.tolist()[0],
                inputs=batch_input.tolist()[0] if save_inputs else None
                )
            data_list.append(data)            

        # Save data to json
        with open(self.conf.pred_val_dataset_json, "w") as f:
            json.dump([ d.to_dict() for d in data_list ], f)
            logger.info(f"Saved data to {f.name}")

        # Reset options
        self.conf.batch_size = batch_size_save
        self.conf.shuffle = shuffle_save

        return data_list


    def train(self):
        """Train the model
        """        

        # Create the optimizer
        optimizer_cls = getattr(torch.optim, self.conf.optimizer)
        optimizer = optimizer_cls(self.model.parameters(), lr=self.conf.learning_rate)

        # Load if needed
        if self.conf.initialize == self.conf.Initialize.FROM_LATEST_CHECKPOINT:
            epoch_start, val_loss_best = self.load_checkpoint(fname=self.conf.checkpoint_latest, optimizer=optimizer)
            data_norm = self.load_data_norm()
        elif self.conf.initialize == self.conf.Initialize.FROM_BEST_CHECKPOINT:
            epoch_start, val_loss_best = self.load_checkpoint(fname=self.conf.checkpoint_best, optimizer=optimizer)
            data_norm = self.load_data_norm()
        elif self.conf.initialize == self.conf.Initialize.FROM_SCRATCH:
            epoch_start, val_loss_best = 0, float("inf")

            # Clear the output directory
            if os.path.exists(self.conf.output_dir):
                logger.warning(f"Output directory {self.conf.output_dir} already exists. Deleting it to start over. You have 8 seconds.")
                for _ in range(8):
                    print(".", end="", flush=True)
                    time.sleep(1)
                print("")
                shutil.rmtree(self.conf.output_dir)
            makedirs(self.conf.output_dir)

            # Save initial weights
            self._save_checkpoint(epoch=epoch_start, optimizer=optimizer, loss=val_loss_best, fname=self.conf.checkpoint_init)
            data_norm = None

            # Copy the config to the output directory for reference
            fname_conf = os.path.join(self.conf.output_dir, "conf.yml")
            makedirs(self.conf.output_dir)
            with open(fname_conf, "w") as f:
                yaml.dump(self.conf.to_dict(), f, indent=3)
                logger.info(f"Saved configuration to {f.name}")
        
        else:
            raise NotImplementedError(f"Initialize {self.conf.initialize} not implemented")
        train_data = self.conf.load_training_metadata_or_new(epoch_start)

        # Create the loaders
        loader_train, loader_val, data_norm = self.conf.create_data_loaders_train_val(data_norm)

        # Write data normalization
        self.conf.write_data_norm(data_norm)

        # Train
        epoch_last_improvement = None
        for epoch in range(epoch_start, self.conf.num_epochs):
            logger.info(f"Epoch {epoch+1}/{self.conf.num_epochs}")
            t0 = time.time()

            # Training
            train_loss = 0
            for batch_input, batch_pred in tqdm(loader_train, desc="Training batches"):
                batch_input, batch_pred = batch_input.to(self.conf.device), batch_pred.to(self.conf.device)
                train_loss += self._train_step(batch_input, batch_pred, optimizer)

            # Validation loss
            self.model.eval()
            with torch.no_grad():
                val_loss = 0
                for batch_input, batch_pred in tqdm(loader_val, desc="Validation batches"):
                    batch_input, batch_pred = batch_input.to(self.conf.device), batch_pred.to(self.conf.device)
                    val_loss += self._compute_loss(batch_input, batch_pred).item()

            # Log
            train_loss /= len(loader_train)
            val_loss /= len(loader_val)
            dur = time.time() - t0
            logger.info(f"Training loss: {train_loss:.5f} val: {val_loss:.5f} duration: {dur:.2f}s")

            # Store metadata about training
            train_data.epoch_to_data[epoch] = TrainingMetadata.EpochData(epoch=epoch, train_loss=train_loss, val_loss=val_loss, duration_seconds=dur)

            # Save checkpoint
            if val_loss < val_loss_best:
                logger.info(f"New best validation loss: {val_loss:.5f}")
                self._save_checkpoint(epoch=epoch, optimizer=optimizer, loss=val_loss, fname=self.conf.checkpoint_best)
                val_loss_best = val_loss
                epoch_last_improvement = epoch
            self._save_checkpoint(epoch=epoch, optimizer=optimizer, loss=val_loss, fname=self.conf.checkpoint_latest)
            self.conf.write_training_metadata(train_data)

            # Early stopping
            if epoch_last_improvement is not None and self.conf.early_stopping_patience is not None and epoch - epoch_last_improvement >= self.conf.early_stopping_patience:
                logger.info(f"Stopping early after {epoch - epoch_last_improvement} epochs without improvement in validation loss.")
                break


    def _save_checkpoint(self, epoch: int, optimizer: torch.optim.Optimizer, loss: float, fname: str):
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, fname)


    def _compute_loss(self, batch_input: torch.Tensor, batch_pred: torch.Tensor) -> torch.Tensor:
        """Compute the loss

        Args:
            batch_input (torch.Tensor): Batch input of shape (batch_size, input_length (time), no_features)
            batch_pred (torch.Tensor): Batch prediction of shape (batch_size, prediction_length (time), no_features)

        Returns:
            torch.Tensor: Loss (MSE)
        """        

        # Forward pass
        batch_pred_hat = self.model(batch_input)

        # Compute MSE loss
        loss = torch.nn.functional.mse_loss(batch_pred_hat, batch_pred)

        # Normalize the loss by the batch size
        # batch_size = batch_input.size(0)
        # loss /= batch_size

        return loss


    def _train_step(self, batch_input: torch.Tensor, batch_pred: torch.Tensor, optimizer: torch.optim.Optimizer) -> float:
        """Training step

        Args:
            batch_input (torch.Tensor): Input data of shape (batch_size, input_length (time), no_features)
            batch_pred (torch.Tensor): Prediction data of shape (batch_size, prediction_length (time), no_features)
            optimizer (torch.optim.Optimizer): Optimizer

        Returns:
            float: Loss (MSE)
        """        
        optimizer.zero_grad()

        # Train mode
        self.model.train()

        # Loss
        loss = self._compute_loss(batch_input, batch_pred)

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        return loss.item()

### Import

In [4]:
from utils.model import TSBatchNorm2d, TSFeatMixingResBlock, TSMixerModelExclRIN, TSMixingLayer, TSTemporalProjection, TSTimeMixingResBlock, TSMixerModel
from utils.plotting import plot_preds, plot_loss
from utils.tsmixer_conf import TSMixerConf, TrainingMetadata
from utils.tsmixer_grid_search_conf import TSMixerGridSearch
from utils.tsmixer import TSMixer

from utils.tsmixer_conf import TSMixerConf, TrainingMetadata, makedirs
from utils.model import TSMixerModel
from utils.load_csv import DataNormalization

import os
from typing import Optional, Tuple, Dict, List, Callable
from loguru import logger
from tqdm import tqdm
from dataclasses import dataclass
from mashumaro import DataClassDictMixin
import yaml
import pandas as pd
from torch.utils.data import DataLoader, Dataset, Subset
from enum import Enum
import torch

### Train

In [19]:
command ="train"
conf_path ="coeficients.yml"

assert conf_path, "Must provide a configuration file"
with open(conf_path, "r") as f:
    conf = TSMixerConf.from_dict(yaml.safe_load(f))
conf.device = "cpu"
tsmixer = TSMixer(conf)
tsmixer.train()

2024-01-26 19:42:32.543 | WARNING  | utils.tsmixer:train:199 - Output directory output.etdataset already exists. Deleting it to start over. You have 8 seconds.


........

2024-01-26 19:42:40.583 | INFO     | utils.tsmixer:train:216 - Saved configuration to output.etdataset\conf.yml
2024-01-26 19:42:40.617 | DEBUG    | utils.load_csv:load_csv_dataset:147 - Computed data mean for each feature: [39.305194812296435, 9.486974868679393, 45.01772580206, 8.773417792042638, -1.6132898368980106, -2.1511628946076273, 27.38195973950271]
2024-01-26 19:42:40.617 | DEBUG    | utils.load_csv:load_csv_dataset:148 - Computed data std for each feature: [9.873003540404081, 6.2120800041681905, 14.235528308701587, 4.456503663368539, 5.6089981540270735, 6.822933637753333, 11.853705964787029]
2024-01-26 19:42:40.619 | DEBUG    | utils.tsmixer_conf:write_data_norm:180 - Saved data normalization to output.etdataset\data_norm.json
2024-01-26 19:42:40.621 | INFO     | utils.tsmixer:train:231 - Epoch 1/30


Validation batches: 100%|██████████| 53/53 [00:01<00:00, 46.10it/s]
2024-01-26 19:42:51.593 | INFO     | utils.tsmixer:train:252 - Training loss: 0.35078 val: 0.17924 duration: 10.97s
2024-01-26 19:42:51.593 | INFO     | utils.tsmixer:train:259 - New best validation loss: 0.17924
2024-01-26 19:42:51.619 | INFO     | utils.tsmixer:train:231 - Epoch 2/30
Validation batches: 100%|██████████| 53/53 [00:01<00:00, 44.81it/s]
2024-01-26 19:43:03.611 | INFO     | utils.tsmixer:train:252 - Training loss: 0.28668 val: 0.17091 duration: 11.99s
2024-01-26 19:43:03.612 | INFO     | utils.tsmixer:train:259 - New best validation loss: 0.17091
2024-01-26 19:43:03.634 | INFO     | utils.tsmixer:train:231 - Epoch 3/30
Validation batches: 100%|██████████| 53/53 [00:01<00:00, 37.77it/s]
2024-01-26 19:43:16.083 | INFO     | utils.tsmixer:train:252 - Training loss: 0.26845 val: 0.16652 duration: 12.45s
2024-01-26 19:43:16.083 | INFO     | utils.tsmixer:train:259 - New best validation loss: 0.16652
2024-01-2

In [9]:
no_feats_plot = 5
conf_path="coeficients.yml"
assert conf_path, "Must provide a configuration file"
with open(conf_path, "r") as f:
    conf = TSMixerConf.from_dict(yaml.safe_load(f))

conf.initialize = TSMixerConf.Initialize.FROM_BEST_CHECKPOINT
conf.device = "cpu"
tsmixer = TSMixer(conf)

data = tsmixer.predict_val_dataset(max_samples=10, save_inputs=False)
data_plt = data[0]
plot_preds(preds=data_plt.pred, preds_gt=data_plt.pred_gt, no_feats_plot=no_feats_plot, fname_save=os.path.join(conf.image_dir, "preds.png"))
plt.close()


2024-01-26 23:31:39.376 | DEBUG    | utils.tsmixer:load_checkpoint:68 - Loading model weights from output.etdataset\best.pth
2024-01-26 23:31:39.419 | INFO     | utils.tsmixer:load_checkpoint:77 - Loaded optimizer state from epoch 4 with loss 0.1647655925941917
2024-01-26 23:31:39.427 | DEBUG    | utils.tsmixer:load_data_norm:110 - Loading data normalization from output.etdataset\data_norm.json


Predicting: 100%|██████████| 10/10 [00:00<00:00, 243.86it/s]
2024-01-26 23:31:39.681 | INFO     | utils.tsmixer:predict_val_dataset:170 - Saved data to output.etdataset\pred_val_dataset.json
2024-01-26 23:31:39.943 | INFO     | utils.plotting:plot_preds:57 - Saved plot to output.etdataset\images\preds.png


NameError: name 'plt' is not defined

In [8]:
conf_path

assert conf_path, "Must provide a configuration file"
with open(conf_path, "r") as f:
    conf = TSMixerConf.from_dict(yaml.safe_load(f))

conf.device = "cpu"
train_data = conf.load_training_metadata_or_new()
plot_loss(train_data=train_data, fname_save=os.path.join(conf.image_dir, "loss.png"))
plt.close


2024-01-26 23:31:04.650 | INFO     | utils.plotting:plot_loss:93 - Saved plot to output.etdataset\images\loss.png


NameError: name 'plt' is not defined